In [10]:
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
from typing import Literal
from pathlib import Path

In [11]:
client = genai.Client(
    vertexai=True,
    location='europe-west4'
)

In [12]:
audio_path = Path("../data/customer_support_call.m4a")

audio = types.Part.from_bytes(
    data=audio_path.read_bytes(),
    mime_type="audio/x-m4a"
)

In [13]:
class CallSummary(BaseModel):
    customer_email: str
    topics_discussed: list[str]
    resolved: bool = Field(description="Whether the customer's problem was resolved.")
    customer_sentiment: Literal[
        "Unhappy",
        "Neutral",
        "Happy",
    ]
    problem: str = Field(description="The customer's problem.")
    root_cause: str
    solution: str

response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=[
        "Find below a SparkleNet Customer Support conversation.",
        audio,
    ],
    config=types.GenerateContentConfig(
        temperature=0.2,
        response_mime_type='application/json',
        response_schema=CallSummary,
    ),
)
call_summary: CallSummary = response.parsed
call_summary

CallSummary(customer_email='john.davis@email.com', topics_discussed=['login issues', 'password reset', 'browser troubleshooting', 'multi-factor authentication'], resolved=True, customer_sentiment='Happy', problem='Customer was unable to log into his SparkleNet account despite resetting his password.', root_cause='The password reset was not fully processed on the system due to a temporary glitch.', solution='The customer support agent manually triggered a new password reset, and the customer was able to log in after resetting his password. The agent also provided information about multi-factor authentication.')

In [14]:
print(call_summary.model_dump_json(indent=2))

{
  "customer_email": "john.davis@email.com",
  "topics_discussed": [
    "login issues",
    "password reset",
    "browser troubleshooting",
    "multi-factor authentication"
  ],
  "resolved": true,
  "customer_sentiment": "Happy",
  "problem": "Customer was unable to log into his SparkleNet account despite resetting his password.",
  "root_cause": "The password reset was not fully processed on the system due to a temporary glitch.",
  "solution": "The customer support agent manually triggered a new password reset, and the customer was able to log in after resetting his password. The agent also provided information about multi-factor authentication."
}


## Cost estimates

In [15]:
response.usage_metadata.total_token_count \
/ 1_000_000 \
* 1.00

0.008406

## Tabular format

In [21]:
import pandas as pd
pd.read_json("../data/customer_service_call_reports.json")

,customer_email,topics_discussed,resolved,sentiment,problem,root_cause,solution
0,alice.smith@email.net,"[billing inquiry, subscription cancellation, r...",True,Neutral,Customer wanted to cancel her subscription and...,Customer was unhappy with the new features and...,Agent cancelled the subscription effective imm...
1,bob.jones@example.org,"[technical support, software installation, com...",False,Unhappy,Customer was unable to install the latest vers...,The customer's computer did not meet the minim...,Agent advised the customer that the software w...
2,jane.doe@company.com,"[account creation, profile settings, security ...",True,Happy,Customer had difficulty setting up her account...,The user interface was not intuitive enough fo...,The agent walked the customer through the acco...
3,peter.pan@neverland.com,"[feature request, product feedback, usability ...",False,Neutral,Customer wanted to suggest a new feature and p...,Customer felt the current workflow was ineffic...,The agent thanked the customer for his feedbac...
4,wendy.darling@neverland.com,"[payment issues, invoice details, incorrect ch...",True,Happy,Customer noticed incorrect charges on her invo...,The system had incorrectly applied a surcharge.,"The agent investigated the issue, identified t..."
...,...,...,...,...,...,...,...
112,henry.moore@creative.com,"[creative cloud, application compatibility, fi...",False,Unhappy,Customer experienced issues with syncing files...,Network connectivity problems and application ...,Agent troubleshooted basic network settings an...
113,georgia.okeefe@creative.com,"[adobe photoshop, image editing, tool function...",True,Happy,Customer had difficulty understanding a specif...,The tool was not intuitive for new users.,Agent provided a detailed explanation of the t...
114,salvador.dali@creative.com,"[adobe illustrator, vector graphics, performan...",True,Neutral,Customer experienced performance issues and la...,Insufficient system resources and large file s...,Agent recommended upgrading system resources a...
115,pablo.picasso@creative.com,"[adobe premiere pro, video editing, export set...",True,Happy,Customer needed assistance with choosing the o...,Customer was unsure of the different codecs an...,Agent guided the customer through various expo...
